# PART1

In [1]:
#데이터 로드
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from dmba import adjusted_r2_score, AIC_score, BIC_score

df = pd.read_csv('part1_to_send/data_1.txt', delimiter='\t')

df

,Price,Age_08_04,KM,HP,Met_Color,Automatic,CC,Doors,Quarterly_Tax,Weight,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Fuel_Type
0,13500,23,46986,90,1,0,2000,3,210,1165,0,1,0,Diesel
1,13750,23,72937,90,1,0,2000,3,210,1165,0,1,0,Diesel
2,13950,24,41711,90,1,0,2000,3,210,1165,0,1,0,Diesel
3,14950,26,48000,90,0,0,2000,3,210,1165,0,1,0,Diesel
4,13750,30,38500,90,0,0,2000,3,210,1170,0,1,0,Diesel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,7500,69,20544,86,1,0,1300,3,69,1025,0,0,1,Petrol
1432,10845,72,19000,86,0,0,1300,3,69,1015,0,0,1,Petrol
1433,8500,71,17016,86,0,0,1300,3,69,1015,0,0,1,Petrol
1434,7250,70,16916,86,1,0,1300,3,69,1015,0,0,1,Petrol


In [2]:
# predictor 와 outcome 설정
predictors_1 = ['KM', 'HP' , 'CC']
predictors_2 = ['KM', 'HP' , 'CC', 'Doors']
predictors_3 = ['KM', 'HP' , 'CC', 'Doors', 'Quarterly_Tax']

outcome = 'Price'

In [3]:
#train_test_split으로 
X_train, X_test, y_train, y_test = train_test_split(df[predictors_3], df[outcome], test_size=0.2, random_state=42)

model_1 = LinearRegression().fit(X_train[predictors_1], y_train)
model_2 = LinearRegression().fit(X_train[predictors_2], y_train)
model_3 = LinearRegression().fit(X_train[predictors_3], y_train)

In [4]:
models = [model_1, model_2, model_3]
predictors = [predictors_1, predictors_2, predictors_3]

for model, predictor in zip(models,predictors):
    y_pred = model.predict(X_train[predictor])
    
    print('adjusted r2 : ', adjusted_r2_score(y_train, y_pred, model))
    print('AIC : ', AIC_score(y_train, y_pred, model))
    print('BIC : ', BIC_score(y_train, y_pred, model))

adjusted r2 :  0.3645866603269027
AIC :  21557.27698940593
BIC :  21582.50587229033
adjusted r2 :  0.3826577517151286
AIC :  21525.150787864954
BIC :  21555.42544732623
adjusted r2 :  0.5292677769645648
AIC :  21214.883953817884
BIC :  21250.20438985604


adjusted R^2, AIC, BIC 모두 Model 3이 제일 좋게 나왔다.

In [5]:
#5-fold cross-validation
from sklearn.model_selection import cross_val_score

for model, predictor in zip(models,predictors):
    
    scores = cross_val_score(model, df[predictor], df[outcome], cv=5)
    
    print(f"Model using {len(predictor)} predictors: Mean Score = {scores.mean()}, Std Deviation = {scores.std()}")

Model using 3 predictors: Mean Score = -3.193879309151827, Std Deviation = 4.356283591466851
Model using 4 predictors: Mean Score = -3.078192776421731, Std Deviation = 4.222937197133012
Model using 5 predictors: Mean Score = -2.5872213564886044, Std Deviation = 3.1220639174135694


5-fold cross-validation의 결과도 Model 3이 가장 좋게 나왔다. (평균과 표준편차의 크기가 가장 작아서)

# PART2

Multivariate regression analysis에선 multi-colinearity가 생길 수 있고, 모델 전체의 signifance와 각각 변수의 signifance에도 영향을 미칠 수 있다. multi-colinearity가 감지되었다면 상관관게가 높은 변수들을 하나만 빼고 제거하면 될 것 같다.

# PART 3

k=3이므로 L-1 distance를 사용하여 (1.5, 1.7) 가장 가까운 주변 점 3개를 찾아야 한다


d(1,1) = |1 - 1.5| + |1 - 1.7| = 0.2
d(1,3) = |1 - 1.5| + |3 - 1.7| = 2.8
d(3,3) = |3 - 1.5| + |3 - 1.7| = 2.8
d(3,1) = |3 - 1.5| + |1 - 1.7| = 1.8
d(3,2) = |3 - 1.5| + |2 - 1.7| = 1.8
d(2,3) = |2 - 1.5| + |3 - 1.7| = 1.8
d(1,2) = |1 - 1.5| + |2 - 1.7| = 0.8
d(2,1) = |2 - 1.5| + |1 - 1.7| = 0.8

(1,1), (1,2), (2,1) 3개가 가장 가깝고, 각각에 해당하는 Y 값은 0, 1, 1 이므로 (1.5,1.7)의 Y값이 1일 확률은 2/3 이다.

# PART 4

In [6]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [7]:
x_train = pd.read_csv('part4_to_send/X_train.txt', header=None, delim_whitespace=True)
y_train = pd.read_csv('part4_to_send/y_train.txt', header=None, delim_whitespace=True)

column_names = pd.read_csv('part4_to_send/inputvariables.txt', header=None,delimiter=' ')



In [8]:
x_train.columns = column_names

In [9]:
x_train

,"(1, tBodyAcc-mean()-X)","(2, tBodyAcc-mean()-Y)","(3, tBodyAcc-mean()-Z)","(4, tBodyAcc-std()-X)","(5, tBodyAcc-std()-Y)","(6, tBodyAcc-std()-Z)","(7, tBodyAcc-mad()-X)","(8, tBodyAcc-mad()-Y)","(9, tBodyAcc-mad()-Z)","(10, tBodyAcc-max()-X)",...,"(552, fBodyBodyGyroJerkMag-meanFreq())","(553, fBodyBodyGyroJerkMag-skewness())","(554, fBodyBodyGyroJerkMag-kurtosis())","(555, angle(tBodyAccMean,gravity))","(556, angle(tBodyAccJerkMean),gravityMean))","(557, angle(tBodyGyroMean,gravityMean))","(558, angle(tBodyGyroJerkMean,gravityMean))","(559, angle(X,gravityMean))","(560, angle(Y,gravityMean))","(561, angle(Z,gravityMean))"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.274893,-0.023347,-0.112283,-0.997806,-0.980717,-0.980161,-0.998243,-0.980349,-0.976160,-0.943916,...,0.453918,-0.529423,-0.824571,-0.006705,0.242755,-0.163498,0.063867,-0.823708,0.211693,0.063784
6996,0.228084,0.004561,-0.028120,-0.960863,-0.741529,-0.714550,-0.962328,-0.752327,-0.708164,-0.937920,...,-0.286429,-0.054585,-0.455079,0.082682,0.010849,0.063542,0.066577,-0.747443,0.115331,-0.164394
6997,0.283156,-0.022120,-0.124696,-0.994269,-0.946728,-0.979608,-0.995052,-0.940315,-0.979405,-0.937920,...,0.439220,-0.619295,-0.884802,0.043939,-0.039754,0.056114,-0.305584,-0.718426,0.104661,-0.186138
6998,0.277953,-0.009818,-0.102730,-0.995815,-0.985390,-0.994174,-0.995805,-0.983275,-0.994116,-0.941224,...,0.617488,-0.784239,-0.941289,-0.070651,0.310339,0.029349,0.006193,-0.724570,0.107792,-0.181549


In [10]:
x_test = pd.read_csv('part4_to_send/X_test.txt', header=None, delim_whitespace=True)

In [11]:
#train, val set 분리
x_train_train, x_train_val, y_train_train, y_train_val = train_test_split(x_train, y_train, test_size=0.4, random_state=42)

In [14]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

k_range = range(3, 9)

param_dist = {'n_neighbors': k_range}
knn = KNeighborsClassifier()
rand_search = RandomizedSearchCV(knn, param_distributions=param_dist, n_iter=6, cv=5, random_state=42)

rand_search.fit(x_train_train, y_train_train.values.ravel())

best_k = rand_search.best_params_['n_neighbors']
print("Best k value:", best_k)

knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(x_train, y_train.values.ravel())

y_test_pred = knn_model.predict(x_test)

np.savetxt('y_test_pred.txt', y_test_pred, fmt='%d')

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Anaconda\lib\s

Best k value: 3


C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


최고의 k는 3인 것으로 나왔다.